In [ ]:
!pip3 install mxnet-mkl==1.6.0 numpy==1.23.1

!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1




# # pip install git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-aqzfccyl/kobert-tokenizer_1f64efe03bc64024ad1fc0a0493b2f73
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-aqzfccyl/kobert-tokenizer_1f64efe03bc64024ad1fc0a0493b2f73
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done


In [ ]:

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [ ]:
# Torch GPU 설정
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

In [ ]:

df = pd.read_csv("/content/drive/MyDrive/sample_dataset.csv")

min_count = df[df['Emotion'].isin(['분노', '슬픔']) == False]['Emotion'].value_counts().min()

grouped_data = df.groupby('Emotion')

data_list = []
for emotion, group in grouped_data:
    sampled_group = group.sample(min_count)
    data_list.append(sampled_group)

data = pd.concat(data_list).reset_index(drop=True)

for emotion in data.Emotion.unique():
    print(f'{emotion}: {data[data["Emotion"] == emotion].shape[0]}')



기쁨: 4830
분노: 4830
불안: 4830
슬픔: 4830
평온: 4830


In [ ]:
# Create a mapping from emotions to numerical values
emotion_mapping = {emotion: idx for idx, emotion in enumerate(data['Emotion'].unique())}

# Add a new column in the dataset with the corresponding numerical label
data['Emotion_Label'] = data['Emotion'].map(emotion_mapping)

# Verify the mapping
print(emotion_mapping)


{'기쁨': 0, '분노': 1, '불안': 2, '슬픔': 3, '평온': 4}


In [ ]:
data_list = []
for q, label in zip(data['Sentence'], data['Emotion_Label']):
    data_list.append([q, label])  # Append sentences and numerical labels

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=23)

print(len(dataset_train), len(dataset_test))


19320 4830


In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

class BERTSentenceTransform:
    def __init__(self, tokenizer, max_seq_length, vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        text_a = line[0]
        tokens_a = self._tokenizer.tokenize(text_a)

        if len(tokens_a) > self._max_seq_length - 2:
            tokens_a = tokens_a[:self._max_seq_length - 2]

        tokens = [self._vocab.cls_token] + tokens_a + [self._vocab.sep_token]
        segment_ids = [0] * len(tokens)

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)
        valid_length = len(input_ids)

        if self._pad:
            padding_length = self._max_seq_length - valid_length
            input_ids += [self._vocab[self._vocab.padding_token]] * padding_length
            segment_ids += [0] * padding_length

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'), np.array(segment_ids, dtype='int32')


In [ ]:

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from torch.utils.data import Dataset

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return len(self.labels)


In [ ]:
# New Hyperparameters
max_len = 100
batch_size = 128  # Increased batch size
warmup_ratio = 0.2  # Increased warmup ratio for smoother learning
num_epochs = 15  # Increased number of epochs for more training iterations
max_grad_norm = 1
log_interval = 200
learning_rate = 3e-5  # Reduced learning rate for more precise updates


In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

# Load the tokenizer for KoBERT
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

# Load the KoBERT model and vocab
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file, padding_token='[PAD]')
    return bertmodel, vocab_b_obj

bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
import gluonnlp as nlp
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)


In [ ]:

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 2)


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5,
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
class EarlyStopping:
    def __init__(self, patience=3, verbose=False, delta=0):

        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''현재 모델을 저장'''
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...")
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss


In [ ]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc



In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()


    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|          | 0/151 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch 1 test acc 0.5660694288913773


  0%|          | 0/151 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

epoch 14 test acc 0.7589804381298992


  0%|          | 0/151 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch 15 test acc 0.7597284434490481


In [ ]:
def predict_emotion(sentence, model, tokenizer, vocab, max_len, device):

    # Preprocess the input sentence
    transform = BERTSentenceTransform(tokenizer, max_seq_length=max_len, vocab=vocab, pad=True, pair=False)
    input_ids, valid_length, segment_ids = transform([sentence])

    # Convert numpy arrays to a single tensor (avoiding UserWarning)
    input_ids = torch.tensor(np.array([input_ids]), dtype=torch.long).to(device)
    segment_ids = torch.tensor(np.array([segment_ids]), dtype=torch.long).to(device)
    valid_length = torch.tensor(np.array([valid_length]), dtype=torch.int32).to(device)

    # Set the model to evaluation mode
    model.eval()

    with torch.no_grad():
        # Get the output from the model
        output = model(input_ids, valid_length, segment_ids)
        # Get the predicted class (label) with the highest score
        predicted_label = torch.argmax(output, dim=1).cpu().numpy()[0]

    # Reverse map to get the emotion label
    reverse_emotion_mapping = {v: k for k, v in emotion_mapping.items()}
    prediction = reverse_emotion_mapping[predicted_label]

    return prediction

# Example usage
input_sentence = "오늘 정말 화가 난다. 아침부터 일이 꼬이더니 결국 폭발하고 말았다. 회사에서 프로젝트에 대해 여러 번 설명했는데, 팀원들은 여전히 내 말을 무시하고 제멋대로 행동했다. 그동안 참았던 불만들이 터져 나왔다. 매번 내가 대신 처리해야 하고, 문제가 생기면 다 내 책임으로 돌아온다. 도대체 왜 내가 항상 이런 상황을 겪어야 하는지 모르겠다. 말해봤자 달라지는 것도 없고, 그저 나만 더 스트레스를 받는 것 같다. 더는 참을 수 없을 것 같다. 정말 화가 나서 폭발하기 직전이다. "
predicted_emotion = predict_emotion(input_sentence, model, tokenizer, vocab, max_len, device)
print(f"Predicted Emotion: {predicted_emotion}")



Predicted Emotion: 분노


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def evaluate_model(model, dataloader, device):

    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length
            label = label.long().to(device)

            output = model(token_ids, valid_length, segment_ids)
            predictions = torch.argmax(output, dim=1)

            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(label.cpu().numpy())

    # Compute the evaluation metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')

    # Get a detailed classification report
    report = classification_report(all_labels, all_preds, target_names=list(emotion_mapping.keys()))

    # Print the report
    print("Classification Report:\n", report)

    # Return metrics as a dictionary
    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

    return metrics

# Example usage on the test data:
metrics = evaluate_model(model, test_dataloader, device)
print(f"Accuracy: {metrics['accuracy']}")
print(f"Precision: {metrics['precision']}")
print(f"Recall: {metrics['recall']}")
print(f"F1 Score: {metrics['f1_score']}")

  0%|          | 0/38 [00:00<?, ?it/s]

Classification Report:
               precision    recall  f1-score   support

          기쁨       0.89      0.86      0.88       989
          분노       0.65      0.61      0.63       941
          불안       0.61      0.69      0.65       923
          슬픔       0.66      0.65      0.65       992
          평온       1.00      0.96      0.98       985

    accuracy                           0.76      4830
   macro avg       0.76      0.76      0.76      4830
weighted avg       0.76      0.76      0.76      4830

Accuracy: 0.7598343685300207
Precision: 0.7646166206852874
Recall: 0.7598343685300207
F1 Score: 0.7615410896298589


In [ ]:

# Define the path where you want to save the model
save_path = "./final_emotion_model.pth"

# Save the trained model's state dictionary
torch.save(model.state_dict(), save_path)

print(f"Model saved at {save_path}")


Model saved at ./final_emotion_model.pth


In [ ]:
# Initialize the model architecture first
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Load the saved model state into the model
model.load_state_dict(torch.load(save_path))

# Don't forget to set the model to evaluation mode if you're using it for inference
model.eval()

print("Model loaded successfully!")


<ipython-input-22-25551d4517f0>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


Model loaded successfully!
